In [ ]:
import pandas as pd
import torch

from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
from pathlib import Path

/home/emanuel/Developer/Personal/learning-ia/machine-learning/prompt-injector-translator/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = "cpu"
BATCH_SIZE = 32
MODEL_NAME = "Helsinki-NLP/opus-mt-en-es"

In [3]:
tokenizer = MarianTokenizer.from_pretrained(pretrained_model_name_or_path=MODEL_NAME)
model = MarianMTModel.from_pretrained(MODEL_NAME).to(DEVICE)

/home/emanuel/Developer/Personal/learning-ia/machine-learning/prompt-injector-translator/.conda/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:176: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Loading weights: 100%|██████████| 258/258 [00:00<00:00, 1199.92it/s, Materializing param=model.shared.weight]                                  
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [4]:
splits = {'train': 'data/train-00000-of-00001-9564e8b05b4757ab.parquet', 'test': 'data/test-00000-of-00001-701d16158af87368.parquet'}

In [5]:
def translate_batch(texts):
    tokens = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_lenght=512
    ).to(DEVICE)

    with torch.no_grad():
        translated = model.generate(**tokens)

    return tokenizer.batch_decode(translated, skip_special_tokens=True)

In [6]:
def translate_column(texts):
    translated_texts = []

    for i in tqdm(range(0, len(texts), BATCH_SIZE)):
        batch = texts[i:i+BATCH_SIZE]
        translated_texts.extend(translate_batch(batch))
    
    return translated_texts

In [ ]:
def process_file(name, input_path):

    df = pd.read_parquet("hf://datasets/JasperLS/prompt-injections/" + input_path)

    assert "text" in df.columns
    assert "label" in df.columns

    texts = df["text"].astype(str).tolist()
    labels = df["label"]

    translated_texts = translate_column(texts)

    translated_df = pd.DataFrame({
        "text": translated_texts,
        "label": labels
    })

    print(translated_df.head())
    print("\n--------")
    print(translated_df.info())
    
    out_path = f"./out/{name}.parquet"

    translated_df.to_parquet(out_path, index=False)

In [ ]:
process_file("train_es", splits["train"])
process_file("test_es", splits["test"])